In [1]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [2]:
project_dir = dirname(@__DIR__)

using JSON: parse

project_json = parse(read(joinpath(project_dir, "project.json"), String))

sample_name = project_json["sample_name"]

output_dir = joinpath(project_dir, "output")

input_dir = joinpath(project_dir, "input")

sample_dir = joinpath(input_dir, sample_name)

reference_dir = joinpath(input_dir, "reference")

snpeff_jar = "/opt/snpeff/snpEff/snpEff.jar"

"/opt/snpeff/snpEff/snpEff.jar"

In [3]:
sample_name

"738"

In [4]:
ProcessSequence.check_program()

Checking program...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


In [5]:
process_dna_arguments = (
    joinpath(
        reference_dir,
        "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz",
    ),
    joinpath(reference_dir, "chromosome.bed.gz"),
    joinpath(reference_dir, "chrn_n.tsv"),
    project_json["n_job"],
    project_json["gb_memory"],
    1,
    snpeff_jar,
)

("/home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chromosome.bed.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chrn_n.tsv", 8, 40, 1, "/opt/snpeff/snpEff/snpEff.jar")

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("germ_dna.1.fastq.gz", "germ_dna.2.fastq.gz")
))

    ProcessSequence.process_germ_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, string("process_germ_dna_", sample_name)),
        process_dna_arguments...,
    )

end

(2020-12-14T04:03:43.596) Trimming sequence ...
`skewer --threads 8 -x AGATCGGAAGAGC --mode pe -Q 2 -q 2 --compress --output /home/jovyan/ProcessSequence.jl/output/process_germ_dna_738/trim_sequence/germ --quiet /home/jovyan/ProcessSequence.jl/input/738/input/WGC056190D_combined_R1.fastq.gz /home/jovyan/ProcessSequence.jl/input/738/input/WGC056190D_combined_R2.fastq.gz`
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	AGATCGGAAGAGC
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- mean quality threshold (-Q):		2
-- end quality threshold (-q):		2
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Solexa/Illumina 1.3+/Illumina 1.5+ FASTQ (auto detected)
-- number of concurrent threads (-t):	8
Mon Dec 14 04:03:43 2020 >> started

Mon Dec 14 07:53:19 2020 >

Started analysis of germ-trimmed-pair1.fastq.gz
Started analysis of germ-trimmed-pair2.fastq.gz
Approx 5% complete for germ-trimmed-pair1.fastq.gz
Approx 5% complete for germ-trimmed-pair2.fastq.gz
Approx 10% complete for germ-trimmed-pair1.fastq.gz
Approx 10% complete for germ-trimmed-pair2.fastq.gz
Approx 15% complete for germ-trimmed-pair1.fastq.gz
Approx 15% complete for germ-trimmed-pair2.fastq.gz
Approx 20% complete for germ-trimmed-pair1.fastq.gz
Approx 20% complete for germ-trimmed-pair2.fastq.gz
Approx 25% complete for germ-trimmed-pair1.fastq.gz
Approx 25% complete for germ-trimmed-pair2.fastq.gz
Approx 30% complete for germ-trimmed-pair1.fastq.gz
Approx 30% complete for germ-trimmed-pair2.fastq.gz
Approx 35% complete for germ-trimmed-pair1.fastq.gz
Approx 35% complete for germ-trimmed-pair2.fastq.gz
Approx 40% complete for germ-trimmed-pair1.fastq.gz
Approx 40% complete for germ-trimmed-pair2.fastq.gz
Approx 45% complete for germ-trimmed-pair1.fastq.gz
Approx 45% complete fo

Analysis complete for germ-trimmed-pair1.fastq.gz
Analysis complete for germ-trimmed-pair2.fastq.gz
(2020-12-14T09:09:27.902) Done in 1 hour, 16 minutes, 8 seconds, 211 milliseconds.
(2020-12-14T09:09:27.921) Aligning sequence ...
pipeline(pipeline(pipeline(pipeline(`minimap2 -x sr -t 8 -K 1G -R '@RG\tID:Germ\tSM:Germ' -a /home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz.mmi /home/jovyan/ProcessSequence.jl/output/process_germ_dna_738/trim_sequence/germ-trimmed-pair1.fastq.gz /home/jovyan/ProcessSequence.jl/output/process_germ_dna_738/trim_sequence/germ-trimmed-pair2.fastq.gz`, stdout=`samtools sort --threads 8 -n`), stdout=`samtools fixmate --threads 8 -m - -`), stdout=`samtools sort --threads 8`), stdout>Base.FileRedirect("/home/jovyan/ProcessSequence.jl/output/process_germ_dna_738/align_sequence/germ.bam.tmp", false))


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::40.342*0.35] loaded/built the index for 195 target sequence(s)
[M::mm_mapopt_update::40.342*0.35] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 195
[M::mm_idx_stat::41.348*0.36] distinct minimizers: 100167746 (38.80% are singletons); average occurrences: 5.519; average spacing: 5.607
[M::worker_pipeline::413.282*5.32] mapped 6741952 sequences
[M::worker_pipeline::628.874*6.30] mapped 6745414 sequences
[M::worker_pipeline::873.858*6.81] mapped 6743388 sequences
[M::worker_pipeline::1074.155*7.08] mapped 6752602 sequences
[M::worker_pipeline::1346.830*7.29] mapped 6744818 sequences
[M::worker_pipeline::1478.940*7.38] mapped 6740830 sequences
[M::worker_pipeline::1654.030*7.46] mapped 6746812 sequences
[M::worker_pipeline::1880.372*7.54] mapped 6752058 sequences
[M::worker_pipeline::2009.630*7.59] mapped 6738114 sequences
[M::worker_pipeline::2260.154*7.6

In [ ]:
if all((
    in(key, keys(project_json))
    for
    key in (
        "germ_dna.1.fastq.gz",
        "germ_dna.2.fastq.gz",
        "soma_dna.1.fastq.gz",
        "soma_dna.2.fastq.gz",
    )
))

    ProcessSequence.process_soma_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_soma_dna"),
        process_dna_arguments...,
    )

end

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("soma_rna.1.fastq.gz", "soma_rna.2.fastq.gz")
))

    ProcessSequence.process_soma_rna(
        joinpath(project_dir, project_json["soma_rna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_rna.2.fastq.gz"]),
        joinpath(output_dir, "process_soma_rna"),
        joinpath(reference_dir, "Homo_sapiens.GRCh38.cdna.all.fa.gz"),
        project_json["n_job"],
    )

end